In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
import pickle

In [ ]:
numbers = [12,3,6,1,7,234,7]
def find_max_range(numbers):
    

In [7]:
def load_data(path: str):
    """
    Function to load nc dataset
    Args:
        path: path to the dataset, includes filename
    Returns:
        dataset: a xarray dataset
    """
    dataset = xr.open_dataset(path)
    return dataset

def return_period(data: np.ndarray ,T : int) -> float:
    """
    Function to calculate return periode
    Args:
        Data: vector of annual max precipitation
        T   : Return period number
    """
    fit_distribution = stats.gumbel_r.fit(data)
    return_period_value = stats.gumbel_r.ppf([1-(1/T)], * fit_distribution)
    
    return  return_period_value
    
def calculate_return_period(data, periods: np.ndarray) -> np.ndarray:
    """
    Function to calculate return period of a matrix data
    Args:
        data: 3D matrix with shape(time, longitude, latitude)
    
    """
    t,x,y = data.shape
    return_period_values = np.zeros((len(periods),x,y))

    for i in range(x):
        for j in range(y):
            for n,T in enumerate(periods):
                return_period_values[n,i,j] = return_period(data[:,i,j], T)
                
    return return_period_values


#array to NC file
def make_nc3D(data_array,periods,lat,lon,var_name,output_name):
    #encode = {parameter: {"zlib":True, "complevel":9}}
    dxr = xr.Dataset(
    {"{}".format(var_name): (("time","longitude", "latitude"), data_array)},
    coords={
        "period" : periods,
        "longitude": lon,
        "latitude": lat,
        })
    dxr.to_netcdf("{}.nc".format(output_name)) #,encoding = encode)

##### Hitung Periode Ulang stasiun

In [3]:
def save_pickle(data,output_name):
    with open(output_name, 'wb') as file:
        pickle.dump(data, file)

def load_pickle(filename):
    with open(filename, 'rb') as file:
        # Use pickle.load() to deserialize and load the data
        loaded_data = pickle.load(file)
    return loaded_data

In [4]:
file_name = "C:/Users/62812/Documents/Kerjaan Meteorologi/GPM-Correction/GPM-Correction/data/all_annual_max.pickle"
annual_max_stasiun = load_pickle(file_name)

In [5]:
for key,val in annual_max_stasiun.items():
    if len(val) < 20:
        print(key,":", len(val), "tahun")

Stasiun Meteorologi Cut Nyak Dhien Nagan Raya : 11 tahun
Stasiun Pemantau Atmosfer Global Bukit Koto Tabang : 8 tahun
Stasiun Meteorologi Perak I : 11 tahun
Stasiun Meteorologi Yuvai Semaring : 16 tahun
Stasiun Meteorologi Fransiskus Xaverius Seda : 16 tahun
Stasiun Meteorologi Gamar Malamo : 15 tahun
Stasiun Meteorologi Wamena Jaya Wijaya : 19 tahun
Stasiun Meteorologi Frans Kaisiepo : 19 tahun


In [6]:
for key,val in annual_max_stasiun.items():
    if np.isnan(np.max(val)):
        print(key)

Stasiun Pemantau Atmosfer Global Bukit Koto Tabang
Stasiun Meteorologi Raja Haji Fisabilillah
Stasiun Meteorologi Yuvai Semaring
Pos Pengamatan Kahang-Kahang
Stasiun Meteorologi David Constatijn Saudale
Stasiun Meteorologi Majene
Stasiun Meteorologi Andi Jemma
Stasiun Meteorologi Syukuran Aminudin Amir
Stasiun Geofisika Manado
Stasiun Meteorologi Mathilda Batlayeri
Stasiun Meteorologi Wamena Jaya Wijaya
Stasiun Meteorologi Enarotali


In [7]:
def calculate_return_period_stations(annual_max,periods):
    pu_stations = {}
    for key,val in annual_max.items():
        try:
            return_period_T = []
            for T in periods:
                return_period_T.append(return_period(val,T)[0])
            pu_stations[key] = return_period_T
        except:
            pass
    return pu_stations

In [8]:
periods = np.arange(2,101)
return_period_stations_indonesia = calculate_return_period_stations(annual_max_stasiun,periods)

In [9]:
save_pickle(return_period_stations_indonesia,"Nilai Periode Ulang Stasiun Indonesia.pickle")

##### Hitung Periode Ulang GPM

In [5]:
path = "C:/Users/62812/Documents/Kerjaan Meteorologi/Data/annual max gpm.nc"

In [9]:
# import time

# start = time.time()

# dataset = load_data(path)
# annual_max = dataset['__xarray_dataarray_variable__'].values
# periods = np.arange(2,101)
# pu_indo = calculate_return_period(annual_max, periods)

# end = time.time()
# print(f"Runtime: {end - start}")

Runtime: 7259.39483499527


In [10]:
filename_output = "Nilai Periode Ulang Indonesia"
latitude = dataset['latitude'].values
longitude = dataset['longitude'].values
var_name = "periode_ulang"

make_nc3D(data_array = pu_indo,
          periods = periods,
          lat = latitude,
          lon = longitude,
          var_name = var_name,
          output_name = filename_output)